In [5]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
from tqdm import tnrange
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
import pickle as pickle

## 01. 기업별 자기소개서 URL 목록 만들기

In [3]:
company_df = pd.DataFrame(columns = ['com_id', 'com_name','urls', 'total_essays'])
company_df

,com_id,com_name,urls,total_essays


In [4]:
# 기업명 및 아이디 추가

def get_company():
    
    com_name = []
    com_id = []
    
    page_url = 'http://www.jobkorea.co.kr/starter/PassAssay'
    web = requests.get(page_url)
    source = BeautifulSoup(web.text, 'html.parser')
    
    groups=source.find('dd',{'class':'lists group30'}).find('div',{'class':'chkScroll'}).find_all('ul')

    for i in range(len(groups)):
        for j in range(len(groups[i].find_all('li',{'class':'itemCheck'}))):
            com_name.append(groups[i].find_all('li',{'class':'itemCheck'})[j].get('data-value'))
            com_id.append(groups[i].find_all('li',{'class':'itemCheck'})[j].get('data-code'))
            
    company_df['com_name'] = com_name
    company_df['com_id'] = com_id

    return('기업 코드가 추가되었습니다')

In [5]:
get_company()

'기업 코드가 추가되었습니다'

In [8]:
company_df.head()

,com_id,com_name,urls,total_essays
0,83,교보생명보험,NaN,NaN
1,31,농협,NaN,NaN
2,2,대림,NaN,NaN
3,7,롯데,NaN,NaN
4,97,메리츠금융,NaN,NaN


In [7]:
# 회사명 내 '그룹' 삭제

company_df['com_name'] = company_df['com_name'].map(lambda x: x[:-2])
company_df.head()

,com_id,com_name,urls,total_essays
0,83,교보생명보험,NaN,NaN
1,31,농협,NaN,NaN
2,2,대림,NaN,NaN
3,7,롯데,NaN,NaN
4,97,메리츠금융,NaN,NaN


In [33]:
# 기업별 합격자소서 링크 크롤링

def company_url(company):

    # 기업명에 해당하는 코드 불러오기
    com_code = company_df['com_id'][company_df['com_name'] == company].values[0]
    
    # url 에 기업 코드 입력하여 리퀘스트 생성
    page_url = 'http://www.jobkorea.co.kr/starter/PassAssay?schGroup={}'.format(com_code)
    web = requests.get(page_url)
    source = BeautifulSoup(web.text, 'html.parser')
    
    # 방문할 페이지 수를 구하기 위해 총 자소서 목록 불러오기
    total = int(source.find('span',{'id':'titSelCount'}).get_text())
    idx_0 = company_df['total_essays'][company_df['com_name'] == company].index[0]
    company_df.iloc[idx_0][3] = total

    # 방문할 페이지 수 구하기
    if total%20==0 :
        max_page = total//20
    else:
        max_page = total//20 +1
    
    # 자소서 url 저장할 리스트 생성
    url_list = []

    # 자소서 url 불러오는 크롤러 생성 
    for page in range(1, max_page + 1): 
        
        compay_url = 'http://www.jobkorea.co.kr/starter/PassAssay?schGroup={}&Page={}'.format(com_code,page)
        web = requests.get(compay_url)
        source = BeautifulSoup(web.text, 'html.parser')  
            
        for i in range(len(source.find('ul',{'class':'selfLists'}).find_all('li'))):
            url = 'http://www.jobkorea.co.kr' + source.find_all('a',{'class':'logo'})[i].get('href')
            url_list.append(url)

    # 기업 urls 열에 리스트 추가
    idx = company_df['com_id'][company_df['com_name'] == company].index[0]
    company_df.iloc[idx][2] = url_list

    return url_list

In [34]:
# 11대 기업 목록 생성
names = ['롯데', '삼성', 'CJ', 'SK', 'LG', 'KT', '한국전력공사', 'GS', '한화', '농협', '현대자동차']

for name in names:
    company_url(name)

In [35]:
# url 리스트 생성된 dataframe

url_df = company_df[company_df['urls'].notnull()]
url_df.reset_index(inplace=True)

In [36]:
url_df

,index,com_id,com_name,urls,total_essays
0,1,31,농협,[http://www.jobkorea.co.kr/starter/PassAssay/V...,69
1,3,7,롯데,[http://www.jobkorea.co.kr/starter/PassAssay/V...,140
2,6,9,삼성,[http://www.jobkorea.co.kr/starter/PassAssay/V...,382
3,15,23,한국전력공사,[http://www.jobkorea.co.kr/starter/PassAssay/V...,233
4,17,27,한화,[http://www.jobkorea.co.kr/starter/PassAssay/V...,98
5,18,29,현대자동차,[http://www.jobkorea.co.kr/starter/PassAssay/V...,345
6,21,34,CJ,[http://www.jobkorea.co.kr/starter/PassAssay/V...,237
7,23,39,GS,[http://www.jobkorea.co.kr/starter/PassAssay/V...,192
8,25,40,KT,[http://www.jobkorea.co.kr/starter/PassAssay/V...,122
9,26,37,LG,[http://www.jobkorea.co.kr/starter/PassAssay/V...,228


In [37]:
# 기업 코드, 기업명 불러오기

company_code_dict = {}

for i, x in enumerate(url_df['com_name']):
    company_code_dict[i] = x
    
company_code_dict

{0: '농협',
 1: '롯데',
 2: '삼성',
 3: '한국전력공사',
 4: '한화',
 5: '현대자동차',
 6: 'CJ',
 7: 'GS',
 8: 'KT',
 9: 'LG',
 10: 'SK'}

In [64]:
# Dataframe 저장: com_id, com_name, urls --> url_df

url_df.to_csv('url_df.csv',index=False)
url_df.to_pickle('url_df.pkl')

# company_code 저장
with open('company_code_dict.pkl', 'wb') as f:
    pickle.dump(company_code_dict, f)

## 02. 개별 자기소개서 내용 불러오기

In [39]:
essays_df = pd.DataFrame(columns = ['company', 'year', 'season', 'experience','position', 'score','feedback','content', 'views'])
essays_df

,company,year,season,experience,position,score,feedback,content,views


In [54]:
company_id = []
company = []
year = []
season = []
experience = []
position = []
score = []
feedback = []
views = []
content = []

error_urls = []

In [55]:
# 기업별 url 리스트에서 url 받아 개별 자기소개서 페이지 방문, 내용 받아오기

# dataframe 에 삽입하기 위한 리스트 생성

def get_many_content(url_list):

    for url in tqdm(range(len(url_list)),desc='scraping', mininterval=1):

        web = requests.get(url_list[url])
        source = BeautifulSoup(web.text, 'html.parser')

        try:
            if i == 0 :
                company_id.append(0)
            else :
                company_id.append(i)
            
            company.append(source.find('h2', {'class': 'hd'}).find('a').get_text())
            info = source.find('h2', {'class': 'hd'}).find('em').get_text()
            year.append(int(info.split()[0][:-1]))
            season.append(info.split()[1])
            experience.append(info.split()[2])
            position.append(info.split()[3])

            views.append(source.find('li', {'class': 'txViewNum'}).get_text()[:-2])

            contents = source.find('div', {'class': 'selfQnaWrap'}).find_all('div', {'class':'tx'})

            string_con = ''
            for essay in contents:
                new_con = essay.get_text()
                new_con = new_con.replace('\n','').replace('\r', '')
                new_con = re.sub('아쉬운점\s\d|글자수+.+Byte|좋은점\s\d', '', new_con)
                string_con = string_con + new_con

            content.append(string_con)

            if source.find('div', {'class' : 'adviceTotal'}) != None:
                score.append(source.find('span', {'class': 'grade'}).get_text(strip=True))
                feedback.append(source.find('div', {'class' : 'adviceTotal'}).find('p').get_text().replace('\n', '').replace('\r', '').replace('[별점이유]', '').replace('[좋은점]', '').replace('[아쉬운점]',''))
                
            else:     
                score.append(None)
                feedback.append(None)

        except:
            error_urls.append(url_list[url])
            return(url_list[url], '에서 오류가 났습니다')

In [56]:
for i in range(11):
    url_list = url_df['urls'][i]
    get_many_content(url_list)


scraping:  75%|███████▍  | 285/382 [09:53<01:45,  1.09s/it]


scraping:   8%|▊         | 28/345 [00:29<05:56,  1.13s/it]


scraping:  81%|████████  | 191/237 [03:01<00:41,  1.10it/s]


scraping:  26%|██▌       | 59/228 [00:58<02:40,  1.05it/s]


scraping: 100%|██████████| 272/272 [03:52<00:00,  1.12it/s]

In [65]:
essay_df = pd.DataFrame({'company_id': company_id,
                          'company': company,
                          'year': year,
                          'season': season,
                          'experience': experience,
                          'position': position,
                          'score': score,
                          'feedback': feedback,
                          'content': content,
                          'views': views})

In [66]:
essay_df

,company_id,company,year,season,experience,position,score,feedback,content,views
0,0,농협은행(주),2020,상반기,신입,사무·총무·법무,2,전반적으로 호감을 주지 못하는 내용 성실히 쓰려고 노력한 점 질문의 의도에 맞지 ...,직장은 생계를 꾸려나가는 수단이기도 하지만 저의 가치관을 실현시킬 수 있는 곳이라고...,"15,387"
1,0,농협은행(주),2019,하반기,신입,금융·보험영업,2,지원자만의 강점이 없음 성실히 쓰려고 노력한 점 금융이나 회사 관련 어떤 준비와 ...,농협은 임직원들이 '농심'. '현장'. '공감'이라는 3대 핵심가치를 기반으로 사고...,"7,360"
2,0,농협은행(주),2019,하반기,신입,금융·보험영업,2,"회사, 직무와 관련하여 지원자만의 강점이 없음 성실히 쓰려고 노력한 점 금융이나 ...","""정확한 의도파악을 통한 문제해결"" 저는 정확한 의도 파악을 통해 윗사람과의 원활한...","4,788"
3,0,농협은행(주),2019,상반기,신입,금융·보험영업,3,지속적으로 금융과 농업에 대해 관심을 가진 것 같아 보이지 않음 하나의 관련 경험...,"""농협, 농업과 더욱 가까이"" 4개월 동안 농협은행 취업아카데미와 농협은행 지역본부...","6,845"
4,0,농협은행(주),2019,하반기,신입,금융·보험영업,2,지원자만의 강점이 없음 성실히 쓰려고 노력한 점 금융이나 회사 관련 어떤 준비와 ...,"""새로운 채널 도전"" 1인 미디어가 주목받는 시대에 새로운 채널에 대한 파악이 늦다...","3,851"
5,0,농협은행(주),2019,하반기,신입,사무·총무·법무,3,"지원회사 및 관련 산업, 직무에 대한 직접적인 준비과정이 없음 질문에 적절한 이...","2015년, ㅇㅇㅇㅇㅇㅇㅇ 소속 사업단으로 'ㅇㅇ ㅇㅇㅇ 식품 박람회'에 참가해, ...","5,470"
6,0,농협은행(주),2019,상반기,신입,금융·보험영업,2,금융영업직과 관련된 내용이 없고 준비과정에 대한 이야기가 없습니다. 성실하게 쓰려...,제가 직장을 선택할 때 가장 우선적으로 생각하는 가치는 함께 일하는 사람과의 인간관...,"4,090"
7,0,농협은행(주),2019,하반기,신입,금융·보험영업,2,전반적으로 호감 가는 내용이 없음 특별히 없음 전반적으로 아쉬움이 많은 자기소개서...,대학생활 중 학생회의 총무부장으로 활동하며 예산을 잡는 데 몇 가지 어려움이 있었습...,"2,124"
8,0,농협은행(주),2019,하반기,신입,금융·보험영업,2,전반적으로 호감 가는 내용이 없음 특별히 좋은 점은 없음 이야기 소재가 식상하고...,"""좌절은 성장의 밑거름"" 갑자기 체중이 불어나 비만이 되고 말았습니다. 거울 속 저...","1,689"
9,0,농협은행(주),2019,하반기,신입,자격증·기술·전문교육,2,전공분야는 연관이 있지만 특별하게 이 직무를 위해 무엇을 준비했는지 잘 모르겠음 ...,"""식물보호기사 필기 합격"" 식물보호기사 1회 필기시험을 준비하면서 개념 위주로 공부...","1,945"


In [67]:
essay_df.to_csv('essay_df.csv',index=False)
essay_df.to_pickle('essay_df.pkl')

## 03. 직무 대분류로 변환 

In [14]:
job_df = pd.DataFrame(columns=['position_broad', 'position'])
job_df

,position_broad,position


In [15]:
wide_range = []
narrow_range = []

page_url = 'http://www.jobkorea.co.kr/starter/PassAssay'
web = requests.get(page_url)
source = BeautifulSoup(web.text, 'html.parser')

wide_jobs=source.find('dd',{'class':'cateSelLists'}).find_all('li')
narrow_jobs = source.find('dd',{'class':'lists'}).find('div',{'class':'nano-content'}).find_all('li')


for j in range(len(wide_jobs)):
    wide_range.append(wide_jobs[j].get('data-value'))
#         narrow_range.append(narrow_jobs[i].get('data-value'))

for i in range(len(narrow_jobs)):
    narrow_range.append(narrow_jobs[i].get('data-value'))

job_df['position'] = narrow_range          
job_df['position_broad'][:9] = wide_range[0]
job_df['position_broad'][9:13] = wide_range[1]             
job_df['position_broad'][13:31] = wide_range[2]
job_df['position_broad'][31:40] = wide_range[3]
job_df['position_broad'][40:49] = wide_range[4]
job_df['position_broad'][49:61] = wide_range[5]
job_df['position_broad'][61:71] = wide_range[6]
job_df['position_broad'][71:79] = wide_range[7]
job_df['position_broad'][79:84] = wide_range[8]
job_df['position_broad'][84:94] = wide_range[9]
job_df['position_broad'][94:101] = wide_range[10]
job_df['position_broad'][101:110] = wide_range[11]
job_df['position_broad'][110:122] = wide_range[12]
job_df['position_broad'][122:] = wide_range[13]

In [21]:
with open('position_wide.pkl', 'wb') as f:
    pickle.dump(wide_range, f)

In [16]:
essay_df = pd.read_csv('essay_df.csv')

In [17]:
essay_df = pd.merge(essay_df, job_df, on='position')

In [18]:
essay_df.head()

,company_id,company,year,season,experience,position,score,feedback,content,views,position_broad
0,0,농협은행(주),2020,상반기,신입,사무·총무·법무,2.0,전반적으로 호감을 주지 못하는 내용 성실히 쓰려고 노력한 점 질문의 의도에 맞지 ...,직장은 생계를 꾸려나가는 수단이기도 하지만 저의 가치관을 실현시킬 수 있는 곳이라고...,"15,387",경영·사무
1,0,농협은행(주),2019,하반기,신입,사무·총무·법무,3.0,"지원회사 및 관련 산업, 직무에 대한 직접적인 준비과정이 없음 질문에 적절한 이...","2015년, ㅇㅇㅇㅇㅇㅇㅇ 소속 사업단으로 'ㅇㅇ ㅇㅇㅇ 식품 박람회'에 참가해, ...","5,470",경영·사무
2,0,농협은행(주),2019,하반기,신입,사무·총무·법무,2.0,회사 및 직무와 관련하여 무엇을 준비하고 노력했는지 알 수 없음 문장이 잘 정리...,글로벌 감각을 키우고 농업에 대한 이해도를 높이기 위한 글로벌 OOO 캠프의 외국인...,"1,591",경영·사무
3,0,농협은행(주),2019,하반기,신입,사무·총무·법무,2.0,지원자가 이 회사에서 무엇을 하고 싶은 지 잘 모르겠음 문장이 잘 정리되어 있음...,"디지털, 산업화로 인한 폐해가 커질수록, 생명에 영향을 끼치는 농업의 중요성이 커지...","2,169",경영·사무
4,0,농협은행(주),2019,하반기,신입,사무·총무·법무,3.0,실제 업무 경험을 사례로 제시한 점은 좋으나 회사 및 직무와 직접적인 연관이 없고...,"자동차 제조사의 금융 제휴 유치를 위해, 협력과 동기부여를 바탕으로 전속 금융사로 ...","4,954",경영·사무


In [20]:
essay_df.to_csv('essay_df_2.csv')
pd.to_pickle(essay_df, 'essay_df_2.pkl')